In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from aiida.orm.data.base import Int, Float, Bool, Str, List
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.folder import FolderData
from aiida.work.run import submit

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import numpy as np

import tempfile
import shutil

from apps.reactions.nebwork import NEBWorkchain

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

# Submit a NEB calculation

## Structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    # Fixed atoms as F
    try:
        bounds = map(int, text_fixed_atoms.value.split('..'))
        for a in range(bounds[0]-1, bounds[1]):
            atoms[a].symbol = 'F'
    except:
        pass
    
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

## Inputs

In [ ]:
from collections import OrderedDict

# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})
qb.order_by({Code: {'id': 'asc'}})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    code = match[1]
    label = match[0]
    all_computers[label] = code

style = {'description_width': '160px'}
layout = {'width': '90%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

int_machines = ipw.IntText(value=1,
                           description='# Nodes per Replica',
                           style=style, layout=layout)

text_calc_name = ipw.Text(placeholder='Some cool name.',
                          description='Calculation Name',
                          style=style, layout=layout)

text_cell = ipw.Text(placeholder='40 40 40',
                     description='Cell (optional)',
                     style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..10',
                            description='Fixed Atoms',
                            style=style, layout={'width': '80%'})

btn_fixed_atoms = ipw.Button(description='apply (F)',
                             layout={'width': '10%'})

btn_fixed_atoms.on_click(lambda b: refresh_structure_view())

# Nodes * Processors/Node * Replicas
int_nproc_rep = ipw.IntText(description='Processors per replica',
                            value='8',
                            style=style, layout=layout)

int_replicas = ipw.IntSlider(description='Replicas',
                             value=7,
                             min=3, max=21,
                             style=style, layout=layout)

float_spring = ipw.FloatText(description='Spring constant',
                             value=0.05,
                             min=0.01, max=0.5, step=0.01,
                             style=style, layout=layout)

toggle_rotate = ipw.ToggleButton(description='Rotate frames',
                                 value=False,
                                 style=style, layout={'width': '45%'})

toggle_align = ipw.ToggleButton(description='Align frames',
                                value=False,
                                style=style, layout={'width': '45%'})

# [how many optimizations before climbing]
int_nstepsit = ipw.IntText(description='NSTEPS_IT',
                           value=1,
                           style=style, layout=layout)

toggle_endpoints = ipw.ToggleButton(description='Optimize endpoints',
                                    value=True,
                                    style=style, layout=layout)

display(
    drop_computer, int_machines,
    text_calc_name, text_cell,
    ipw.HBox([text_fixed_atoms, btn_fixed_atoms]),
    int_nproc_rep, int_replicas, float_spring,
    ipw.HBox([toggle_rotate, toggle_align]),
    int_nstepsit,
    toggle_endpoints
    
)

In [ ]:
toggle_dft = ipw.ToggleButtons(options=['Mixed DFTB', 'Mixed DFT', 'Full DFT'],
                               description='Calculation Type',
                               style=style, layout=layout)

display(toggle_dft)

## Replicas
`Give the replica IDs in the right order initial->final.`

In [ ]:
#rrrrrrrr = '236 237 238 239 240 241 242 243 244 245'
#cell = '58.959978 40.848671 48.118239'

In [ ]:
text_replica_pks = ipw.Text(placeholder='100 105 113 140',
                            description='Replica pks',
                            style=style, layout=layout)

display(text_replica_pks)

## Submission

In [ ]:
def get_slab_atoms(structure):
    atoms = structure.get_ase()
    try:
        first_slab_atom = np.argwhere(atoms.numbers == 79)[0, 0] + 1
        is_H = atoms.numbers[first_slab_atom-1:] == 1
        is_Au = atoms.numbers[first_slab_atom-1:] == 79
        assert np.all(np.logical_or(is_H, is_Au))
    except AssertionError:
        raise Exception("Structure is not a proper slab.")
    
    # the molecule must come before the slab
    return (first_slab_atom, len(atoms))

In [ ]:
def mk_coord_files(structures):
    fd = FolderData()

    tmpdir = tempfile.mkdtemp()
    # We need mol0.xyz for the initial mixed force_eval
    atoms = structures[0].get_ase()
    mol_fn = tmpdir + '/mol0.xyz'
    mol = atoms[:first_slab_atom-1]
    mol.write(mol1_fn)

    # And we also write all the replicas up to the final geometry.
    for i, s in enumerate(structures):
        print i
        atoms = s.get_ase()
        molslab_fn = tmpdir + '/replica{}.xyz'.format(i+1)
        atoms.write(molslab_fn)

    fd.replace_with_folder(folder=tmpdir)
    shutil.rmtree(tmpdir)
    return fd

In [ ]:
#from apps.reactions.nebwork import NEBWorkchain
reload(apps.reactions.nebwork)

def on_submit(b):
    if not drop_computer.value:
        print('Please select a computer.')
        return 0

    if len(text_calc_name.value) < 2:
        print('Please specify a calculation name')
        return 0
    
    replica_pks = text_replica_pks.value.split()
    if len(replica_pks) < 2:
        print('Please select at least two geometries')
        return 0
    else:
        replica_list = [load_node(int(x)) for x in replica_pks]

    if 'Mixed' in toggle_dft.value:
        print('Mixed calculation.')
        # TODO: Check that all replicas have the same number of atoms
        first_slab_atom, last_slab_atom = get_slab_atoms(replica_list[0])
    else:
        first_slab_atom, last_slab_atom = 0, 0
    
    fd = mk_coord_files(replica_list)

    cp2k_code = drop_computer.value
    num_machines = Int(int_machines.value)
    calc_name = Str(text_calc_name.value)
    cell = Str(text_cell.value)
    fixed_atoms = Str(text_fixed_atoms.value)

    nproc_rep = Int(int_nproc_rep.value)
    nreplicas = Int(int_replicas.value)
    spring = Float(float_spring.value)
    rotate = Bool(toggle_rotate.value)
    align = Bool(toggle_align.value)
    nstepsit = Int(int_nstepsit.value)
    endpoints = Bool(toggle_endpoints.value)
    calc_type = Str(toggle_dft.value)

    outputs = submit(apps.reactions.nebwork.NEBWorkchain,
                     cp2k_code=cp2k_code,
                     struc_folder=fd,
                     num_machines=num_machines,
                     calc_name=calc_name,
                     cell=cell,
                     fixed_atoms=fixed_atoms,
                     nproc_rep=nproc_rep,
                     nreplicas=nreplicas,
                     spring=spring,
                     rotate=rotate,
                     align=align,
                     nstepsit=nstepsit,
                     endpoints=endpoints,
                     calc_type=calc_type,
                     first_slab_atom=Int(first_slab_atom),
                     last_slab_atom=Int(last_slab_atom))

    print(outputs)
    the_workfunction = load_node(outputs.pid)
    the_workfunction.description = calc_name

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)